In [25]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import urllib.request
import time
import io
import pandas as pd
from time import gmtime, strftime, localtime

In [27]:
# функция для просмотра страниц магазина/пользователя и сохранения информации об объявлениях
# изображения из объявлений сохраняются в папку, содержащую Avito_crawler.ipynb
# информация об объявлениях сохраняется в xlsx. файл

def avito_crawler(pages):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    res1=[]
    res2=[]
    res3=[]
    res4=[]
    res5=[]
    total=10
    core_link = ("https://www.avito.ru/trade") #адрес магазина
    driver.get(core_link)
    for n in range(pages):
        count = len(driver.find_elements_by_xpath("//a[@class='item-description-title-link']"))
        for m in range(count):
            main_link = driver.find_elements_by_xpath("//a[@class='item-description-title-link']")[m].get_attribute("href")
            driver.get(main_link)
            time.sleep(0.5)
            name = driver.find_element_by_xpath("//meta[@property = 'og:title']").get_attribute("content")
            print ('This is page {} ad №{} {}'.format(n+1,m,name))
            print ("Name extracted...")
            price = driver.find_element_by_xpath("//span[@class='js-item-price']").get_attribute("content")
            print ("Price extracted...")
            html_doc = urllib.request.urlopen(main_link).read()
            soup = BeautifulSoup(html_doc)
            soupresult = soup.findAll('p')
            description=[]
            for desc in soupresult:
                description.append(desc.get_text())
            print ("Description extracted...")
            try:
                driver.find_elements_by_xpath("//li[@data-type='image']")[0]
            except:  
                image_link = driver.find_element_by_xpath("//img[contains(@alt,'фотография')]").get_attribute("src")
                image_name = driver.find_element_by_xpath("//img[contains(@alt,'фотография')]").get_attribute("alt")
                image_name = image_name.translate({ord(i): None for i in '\/?:*"><|'})
                urllib.request.urlretrieve(image_link, "Page{} ad{} {}.jpg".format(n+1,m,image_name))
            for l in range(total):
                    try:
                        driver.find_elements_by_xpath("//li[@data-type='image']")[l+1].click()
                        image_link = driver.find_element_by_xpath("//img[contains(@alt, 'фотография №{}')]".format(l+1)).get_attribute("src")
                        image_name = driver.find_element_by_xpath("//img[contains(@alt, 'фотография №{}')]".format(l+1)).get_attribute("alt")
                        image_name = image_name.translate({ord(i): None for i in '\/?:*"><|'})
                        urllib.request.urlretrieve(image_link, "Page{} ad{} {}.jpg".format(n+1,m,image_name))
                    except:
                        pass
            print ("Images extracted...")
            res1.append(n+1)
            res2.append(m)
            res3.append(name)
            res4.append(price)
            res5.append(description)
            driver.back()
        if n<(pages-1):
            driver.find_element_by_xpath("//a[@class='pagination-page js-pagination-next']").click()  
        else:
            print("Reached final page")
    driver.close()
    keys = ("Номер страницы","Номер объявления","Заголовок","Цена","Описание")
    values = (res1,res2,res3,res4,res5)
    new_dict = dict(zip(keys, values))
    print ("Creating database...")
    df = pd.DataFrame(data=new_dict)
    df['Описание'] = [','.join(map(str, l)) for l in df['Описание']]
    print ('Extracting to Excel...')
    currenttime = strftime("%Y-%m-%d %H.%M.%S", localtime())
    df.to_excel("Avito base {}.xlsx".format(currenttime))
    print ("Task completed")

In [30]:
# в качестве аргумента функции передается количество страниц, которое нужно просмотреть
avito_crawler(1) 


Looking for [chromedriver 81.0.4044.138 mac64] driver in cache 
File found in cache by path [/Users/olegkandybko/.wdm/drivers/chromedriver/81.0.4044.138/mac64/chromedriver]
Reached final page
Creating database...
Extracting to Excel...
Task completed
